In [ ]:
import os
from typing import TypedDict, Annotated, List, Optional, Dict, Any
from dotenv import load_dotenv
import requests

from langchain.memory import ConversationWindowBufferMemory
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import AIMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph import add_messages

In [ ]:
# Load environment variables
load_dotenv()
CYFUTURE_API_KEY = os.getenv("CYFUTURE_API_KEY")

In [ ]:
# Define state with window buffer memory
class State(TypedDict):
    messages: Annotated[list, add_messages]
    intent: str
    city: str
    memory: ConversationWindowBufferMemory

def initialize_memory():
    return ConversationWindowBufferMemory(
        k=5,  # Remember last 5 interactions
        memory_key="chat_history",
        return_messages=True
    )

In [ ]:
def ask_ai(question: str) -> str:
    """Simple function to ask CyfutureAI a question"""
    try:
        response = requests.post(
            "https://api.cyfuture.ai/v1/chat/completions",
            headers={"Authorization": f"Bearer {CYFUTURE_API_KEY}"},
            json={
                "model": "llama8",
                "messages": [{"role": "user", "content": question}]
            }
        )
        return response.json()["choices"][0]["message"]["content"]
    except:
        return "Sorry, I couldn't process that request."

In [ ]:
def classify_intent(state: Dict[str, Any]) -> Dict[str, Any]:
    """Determine if the user is asking for weather data or general information"""
    last_message = state["messages"][-1].content

    prompt = f"""
    Determine the intent of the following message:
    \"{last_message}\"

    If the message is asking about current weather, weather forecast, or weather conditions for a specific location, respond with "weather_api".
    If the message is asking about general weather information, climate patterns, or safety measures, respond with "rag".

    Just respond with either "weather_api" or "rag".
    """

    intent = ask_ai(prompt).strip().lower()

    # Initialize city with a default value
    city = "unknown"

    if "weather_api" in intent:
        city_prompt = f"""
        Extract the city name from this message: \"{last_message}\"
        If no city is mentioned, respond with "unknown".
        Just provide the city name or "unknown", nothing else.
        """
        city_response = ask_ai(city_prompt).strip()
        if city_response.lower() != "unknown":
            city = city_response

    return {"intent": intent, "city": city}